## Import

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import cv2
import json
import numpy as np
from utils import *

In [ ]:
img_name="timecity_01886_none"
img = cv2.imread(f'./svm_for_shadow_generation/images/{img_name}.jpg')
org_mask = cv2.imread(f'./svm_for_shadow_generation/masks/{img_name}.jpg')
data = json.load(open(f'./svm_for_shadow_generation/leg_pos/{img_name}.json', "r"))
fixed_points = np.zeros((1,4,2)) # leg positions
for i in range(4):
    fixed_points[0][i] = data['shapes'][i]['points'][0]

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
background=cv2.subtract(img,org_mask)
cut_obj=cv2.subtract(img,background)

In [ ]:
alpha = create_ellipse_alpha_mask(org_mask, fixed_points, -2, 2,
                                  speed=0.1, overflow=1.2, offset = 90)
revert_shadow_mask_f = np.zeros_like(org_mask).astype(float)
img_f = img.astype(float)

# Multiply the foreground with the alpha matte
foreground_shadow = cv2.multiply(alpha, revert_shadow_mask_f)

# Multiply the background with ( 1 - alpha )
background_img = cv2.multiply(1.0 - alpha, img_f)

# Add the masked foreground and background.
shadow_on_top = cv2.add(foreground_shadow, background_img)
shadow_on_top = shadow_on_top.astype('uint8')

final_background=cv2.subtract(shadow_on_top,org_mask)
final_img = cut_obj+final_background
# plt.imshow(shadow_on_top[...,::-1])
plt.axis('off')
plt.imshow(final_img[...,::-1])

idx = "01"
cv2.imwrite(f'{img_name}_{idx}_merge.png', final_img)
# cv2.imwrite(f'{img_name}_{idx}_alpha.png', alpha*255)
# cv2.imwrite(f'{img_name}_{idx}_shadow_mask.png', pad_warped_shadow_mask)